In [1]:
import os
os.chdir("../")
%pwd

'd:\\DE\\ML OPs\\DSusingMLOps'

In [11]:
os.environ['MLOPS_TRACKING_URI']="https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow"
os.environ['MLOPS_TRACKING_USERNAME']="bibhumlai"
os.environ['MLOPS_TRACKING_PASSWORD']="02c1cfda74cc82affcae200880d030400e561596"


In [3]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd

In [4]:
@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [6]:
class ConfigurationManager():
    def __init__(self, config_filepath=CONFIG_FILE_PATH, 
                                params_filepath= PARAMS_FILE_PATH,
                                schema_filepath=SCHEMA_FILE_PATH ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_eval_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(config.test_data_path),
            model_path=Path(config.model_path),
            all_params=params,
            metric_file_name=Path(config.metric_file_name),
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow")
       
        return model_eval_config

In [7]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
from src.datascience import logger
import mlflow.sklearn
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, predicted) -> dict:
        rmse = np.sqrt(mean_squared_error(actual, predicted))
        mae = mean_absolute_error(actual, predicted)
        r2_square = r2_score(actual, predicted)
        return  rmse, mae, r2_square
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        logger.info(f"Test data shape: {test_data.shape}")
        model = joblib.load(self.config.model_path)
        logger.info(f"Model: {model}")
        test_x= test_data.drop(columns=[self.config.target_column], axis=1)
        test_y= test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2_square) = self.eval_metrics(test_y, predicted_qualities)
            logger.info(f"Elasticnet model (alpha={self.config.all_params['alpha']}, l1_ratio={self.config.all_params['l1_ratio']})")
            logger.info(f"  RMSE: {rmse}")
            logger.info(f"  MAE: {mae}")
            logger.info(f"  R2: {r2_square}")
            
            ##SAVE METRICS
            scores = ({"rmse": rmse, "mae": mae, "r2_square": r2_square})
            save_json(path=Path(self.config.metric_file_name), data= scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_param("alpha", self.config.all_params['alpha'])
            mlflow.log_param("l1_ratio", self.config.all_params['l1_ratio'])
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2_square", r2_square)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model")
            else:
                mlflow.sklearn.log_model(model, "model")

In [ ]:
import mlflow
import dagshub

print("Tracking URI:", mlflow.get_tracking_uri())
print("Experiment list:", mlflow.search_experiments())


In [ ]:
import dagshub
dagshub.init(repo_owner='bibhumlai', repo_name='DSprojectusingMLOps', mlflow=True)
import mlflow
with mlflow.start_run():
  # Your training code here...
  mlflow.log_metric('accuracy', 42)
  mlflow.log_param('Param name', 'Value')

In [24]:
import mlflow

mlflow.set_registry_uri("https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow")
mlflow.set_experiment("test-exp")

with mlflow.start_run():
    mlflow.log_param("test_param", 123)
    mlflow.log_metric("test_metric", 0.95)

2025/09/14 22:42:17 INFO mlflow.tracking.fluent: Experiment with name 'test-exp' does not exist. Creating a new experiment.


🏃 View run dapper-robin-308 at: https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow/#/experiments/1/runs/f6c410aa016941c0a4f21eea9ad8b8cf
🧪 View experiment at: https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow/#/experiments/1


In [27]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    logger.info(f"Model Evaluation Config: {model_eval_config}")
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    logger.exception(e)
    raise e    

[2025-09-14 22:43:11,635: INFO : common: yaml file: config\config.yaml loaded sucessfully]
[2025-09-14 22:43:11,637: INFO : common: yaml file: params.yaml loaded sucessfully]
[2025-09-14 22:43:11,639: INFO : common: yaml file: schema.yaml loaded sucessfully]
[2025-09-14 22:43:11,640: INFO : common: created directory at path :artifacts]
[2025-09-14 22:43:11,641: INFO : common: created directory at path :artifacts/model_evaluation]
[2025-09-14 22:43:11,642: INFO : 1899718429: Model Evaluation Config: ModelEvaluationConfig(root_dir=WindowsPath('artifacts/model_evaluation'), test_data_path=WindowsPath('artifacts/data_transformation/test.csv'), model_path=WindowsPath('artifacts/model_trainer/model.joblib'), all_params=ConfigBox({'alpha': 0.2, 'l1_ratio': 0.1}), metric_file_name=WindowsPath('artifacts/model_evaluation/metrics.json'), target_column='quality', mlflow_uri='https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow')]
[2025-09-14 22:43:11,645: INFO : 1304064896: Test data shape: (

2025/09/14 22:43:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run languid-conch-765 at: https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow/#/experiments/1/runs/9cf3393e12f649b2866bbeef6ed5d7e4
🧪 View experiment at: https://dagshub.com/bibhumlai/DSprojectusingMLOps.mlflow/#/experiments/1
[2025-09-14 22:43:17,811: ERROR : 1899718429: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}]
Traceback (most recent call last):
  File "C:\Users\bibhu\AppData\Local\Temp\ipykernel_1888\1899718429.py", line 6, in <module>
    model_eval.log_into_mlflow()
    ~~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\bibhu\AppData\Local\Temp\ipykernel_1888\1304064896.py", line 43, in log_into_mlflow
    mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetWineModel")
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\DE\ML OPs\DSusingMLOps\venv\Lib\site-packages\mlflow\sklearn\__init__.py", line 426, in log_model
    return Model.log(
           ~

RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}